In [90]:
import json
import csv
import numpy as np

with open('../zarascraper/zara-data/unique-products.json', 'r') as file:
    article_jsons = json.load(file)

with open('../bert/bert_embeddings.csv', mode='r', newline='') as file:
    article_reader = csv.reader(file, delimiter=',')
    article_vectors = [np.array(row) for row in article_reader][1:-1]
    
print(json.dumps(article_jsons[37], indent=4))
print(article_vectors[37])

{
    "@context": "https://schema.org/",
    "@type": "Product",
    "name": "SLINGBACK PIECE HEELS",
    "brand": "ZARA",
    "sku": "410569873-800-35",
    "mpn": "410569873-800-35",
    "description": "Sling back shoes with faux patent leather effect. Thin heel. Front metal detail. Back strap with elastic. Pointed toe. \n\nHeel height: 2.4 inches (6 cm)",
    "image": "https://static.zara.net/assets/public/b4e1/8ac2/34d645a789b7/3416eada28a7/12222510800-p/12222510800-p.jpg?ts=1737714456876&w=1920",
    "color": "Black",
    "size": "5",
    "offers": {
        "@type": "Offer",
        "priceCurrency": "USD",
        "price": "35.99",
        "url": "https://www.zara.com/us/en/slingback-piece-heels-p12222510.html",
        "itemCondition": "https://schema.org/NewCondition"
    }
}
['37' '-0.6471915' '-0.59036815' '0.09469482' '-0.17576599' '0.2645034'
 '0.5662038' '0.72154164' '0.22894093' '-0.33647576' '-0.095982105'
 '0.38042158' '-0.0063101817' '-0.06412193' '0.3399671' '-0.09572

In [ ]:
# FIXME: For the time being, all articles are considered unisex


In [ ]:
# prodName varchar(255) not null,
# prodGender int,
# prodComp varchar(255),
# prodColor varchar(255),
# prodCategory varchar(255),
# prodOrigin varchar(255)

# insert into proddesc (prodName, prodGender, prodColor, prodOrigin)
#     values ("NAME", 2, "COLOR", "COMPANY")

with open('database_queries.txt', mode='w') as sql_file:

    id_value = 1 
    for line in article_jsons:
        # Insertion to prodDesc
        insert_string_desc = f"""INSERT INTO proddesc (prodID, prodName, prodGender, prodColor, prodOrigin)
            VALUES (
                {id_value},
                "{line['name']}",
                2,
                "{line['color']}",
                "{line['brand']}"
            );
            """
        sql_file.write(insert_string_desc + "\n")

        # Insertion to prodPrice
        available_status = 0
        try:
            available = line['offers']['availability']
            if (available == "https://schema.org/InStock"):
                available_status = 1
        except KeyError:
            available_status = 0
            
        # FIXME: For now, anything besides "instock" is considered out of stock
        insert_string_price = f"""INSERT INTO prodprice (prodID, prodInStock, prodRegPrice)
            VALUES (
                {id_value},
                {available_status},
                {line['offers']['price']}
            );
            """
        sql_file.write(insert_string_price + "\n")

        # Insertion into prodVec
        # NOTE: vectors are 0-indexed
        product_vector = article_vectors[id_value - 1]
        vec_position_idx = 0

        insert_string_vector = f"""INSERT INTO prodvec (prodID, vecPos, vecVal)
            VALUES """ 

        for vec_value in product_vector:
            vector_insert = f"({id_value}, {vec_position_idx}, {product_vector[vec_position_idx]}),\n"
            insert_string_vector += vector_insert
            vec_position_idx += 1
        insert_string_vector = insert_string_vector[0:-2] + ";"
        sql_file.write(insert_string_vector + "\n")

        # Insertion into prodLink
        insert_string_link = f"""INSERT INTO prodlink (prodID, prodLink, prodImageLink)
            VALUES (
                {id_value},
                "{line['image']}",
                "{line['offers']['url']}"
            );
            """
        sql_file.write(insert_string_link + "\n")

        id_value += 1
        # NOTE: Uncomment this with a desired threshhold to only import a certain number of articles.
        # Best for testing, as loading all 166 articles in can take up to 10 minutes
        # if (id_value >= x):
        #   break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
